Lecture notes

- Sample size: n
- Predictor dim: p
- Class numbers: k


1. Logistic regression
- logistic function p(X)
- log-odds (logit) is linear in X
- likelihood function: finding coefficients that maximize it. Number of coeff depends on predictor dim
- z-stats

2. Multiple logistic regression

3. Discriminant analysis
- Bayes' theorem
- posterior prob. p_k(x) depends on pi_k (prior prob., easy to compute from Y), and f_k(x) (desity function of X, hard to get from X, but can assume simple forms)
- when p=1
    - assume f_k(x,mu_k,sigma) are Gaussian with the same variance
    - the max of p_k(x) in k <=> the max of discriminant functions in k that is linear in x, which still depends on mu_k, sigma and pi_k
    - need estimate pi_k, mu_k and sigma from X and Y
    - Set pairs of discriminant functions equal to each other to determine Bayes decision boundary
    - Once have k for given x, we can compute p_k(x) for the probability
- when p>1
    - assume f_k(x,mu_k,Sigma) are Gaussian with the same covariance
    - algorithm same as p=1 case
- Forms of discriminant analysis
    - Linear: f_k(x) are Gaussian having the same covariance
    - Quatratic: f_k(x) are Gaussian having different covariance
    - Naive Bayes: X are independent in each class (covariance matrix is diagonal); useful for large p; useful for mixed feature vectors

4. Evaluate threshold value
- Confusion matrix
- True/False positive/negative
- Two types of error: True/False postitive rates
- The Total Error is a weighted average of the False Positive Rate and False Negative Rate. The weights are determined by the Prior Probabilities of Positive and Negative Responses.
- Visual method: ROC+AUC